In [1]:
import pandas as pd

In [2]:
df_categories = pd.read_csv('../data/v1/WikiCSSH_categories.csv')
df_categories.head()

,category,level
0,Computer_science,1
1,Mathematics,1
2,Information_science,1
3,Computer_engineering,1
4,Statistics,1


In [3]:
df_category_links = pd.read_csv('../data/v1/WikiCSSH_category_links.csv')
df_category_links.head()

,parent_cat,child_cat,parent_level,child_level
0,Computer_engineering,Electronic_engineering,1,2
1,Computer_engineering,Software_engineering,1,2
2,Computer_engineering,Computer_hardware,1,2
3,Computer_engineering,Computer_systems,1,2
4,Computer_engineering,Computer_engineering_stubs,1,2


In [4]:
isolate_cats = (
    set(df_categories.category.values) 
    - set(
        set(df_category_links.parent_cat.values) 
        | set(df_category_links.child_cat.values)
    )
)
len(isolate_cats)

91

In [5]:
root_child_cats = set(
    set(df_category_links.parent_cat.values) 
    - set(df_category_links.child_cat.values)
)
len(root_child_cats)

76

In [6]:
df_category_links_all = pd.concat([
    df_category_links,
    df_categories[df_categories.category.isin(root_child_cats)].rename(columns={
        "category": "child_cat", 
        "level": "child_level", #	parent_level	child_level
    }).assign(parent_cat="<ROOT>", parent_level=0),
    pd.DataFrame({
        "parent_cat": [""],
        "child_cat": ["<ROOT>"],
        "parent_level": [-1],
        "child_level": [0],
    })
#     df_categories[df_categories.category.isin(isolate_cats)].rename(columns={
#         "category": "child_cat", 
#         "level": "child_level", #	parent_level	child_level
#     }).assign(parent_cat="", parent_level=0)
    
], axis=0, sort=True).sort_values(["parent_level", "child_level"])
df_category_links_all.shape

(8400, 4)

In [7]:
df_category_links_all.head()

,child_cat,child_level,parent_cat,parent_level
0,<ROOT>,0,,-1
0,Computer_science,1,<ROOT>,0
1,Mathematics,1,<ROOT>,0
2,Information_science,1,<ROOT>,0
3,Computer_engineering,1,<ROOT>,0


In [8]:
df_category_links_all.to_csv('../data/v1/WikiCSSH_category_links_all.csv', index=False)

In [9]:
df_categories[df_categories.level<=5].to_csv("../data/v1/Wikicssh_core_categories.csv", index=False)